## Tutorial: Create an Isolated VLAN and Servers

This notebook demonstrates how to create isolated VLANs and attach servers.

#### Configure the Environment

In [1]:
export OS_REGION_NAME=CHI@UC
export OS_PROJECT_NAME=CH-822154

NODE_TYPE=compute_haswell
KEYPAIR_NAME="my_chameleon_key"
LEASE_NAME="${USER}Lease"
SERVER_NAME="${USER}Server"
NETWORK_NAME=${USER}"Net"
SUBNET_NAME=${USER}"Subnet"
ROUTER_NAME=${USER}"Router"
DEFAULT_CIDR=192.168.1.0/24

#### Create a Lease

In [5]:
PUBLIC_NETWORK_ID=$(openstack network show public -f value -c id)

blazar lease-create \
  --physical-reservation min=1,max=1,resource_properties="[\"=\", \"\$node_type\", \"$NODE_TYPE\"]" \
  --reservation "resource_type=virtual:floatingip,network_id=$PUBLIC_NETWORK_ID,amount=1" \
  --start-date "$(date +'%Y-%m-%d %H:%M' -d'+1 minute')" \
  --end-date "$(date +'%Y-%m-%d %H:%M' -d'+1 day')" \
  "$LEASE_NAME"

bash: 2021-03-02: command not found
bash: 2021-03-03: command not found
usage: blazar lease-create [-h] [-f {json,shell,table,value,yaml}] [-c COLUMN]
                           [--noindent] [--prefix PREFIX]
                           [--max-width <integer>] [--fit-width]
                           [--print-empty] [--start-date START]
                           [--end-date END] [--before-end-date BEFORE_END]
                           [--physical-reservation <min=int,max=int,hypervisor_properties=str,resource_properties=str,before_end=str,on_start=str>]
                           [--reservation <key=value>]
                           [--event <event_type=str,event_date=time>]
                           LEASE
blazar lease-create: error: argument --start-date: expected one argument


: 127

In [ ]:
wait_lease "$LEASE_NAME"

#### Create the Network

In [ ]:
openstack network create --max-width 80 \
                         --provider-network-type vlan \
                         --provider-physical-network physnet1 \
                         $NETWORK_NAME

#### Configure the Network

In [ ]:
openstack subnet create --max-width 80 \
                        --subnet-range $DEFAULT_CIDR \
                        --network $NETWORK_NAME \
                        $SUBNET_NAME
                        
openstack router create --max-width 80 \
                        $ROUTER_NAME
                        
openstack router add subnet $ROUTER_NAME $SUBNET_NAME

#### Start the Server

In [ ]:
openstack server create --max-width 80 \
                        --key-name $KEYPAIR_NAME \
                        $(lease_server_create_default_args "$LEASE_NAME") \
                        $SERVER_NAME

#### Associate a Floating IP

In [ ]:
FLOATING_IP=$(lease_list_floating_ips "$LEASE_NAME" | head -n1)

openstack server add floating ip "$SERVER_NAME" "$FLOATING_IP" \
  && echo "Attached floating ip $FLOATING_IP!"

#### Delete Server

In [ ]:
openstack server delete $SERVER_NAME

#### Deconfigure the Network

In [ ]:
openstack router remove subnet $ROUTER_NAME $SUBNET_NAME
openstack router delete $ROUTER_NAME
openstack subnet delete $SUBNET_NAME
openstack network delete $NETWORK_NAME

#### Release Lease

In [ ]:
blazar lease-delete $LEASE_NAME